In [3]:
from EEGModels import EEGNet
import scipy
import numpy as np
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import balanced_accuracy_score

In [4]:
folder_path = './execution_data_clean/'
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.mat')]

file_list.sort()

print(file_list)

X = []
Y = []

for i, file in enumerate(file_list):
    x = scipy.io.loadmat(file)
    x = x['data']

    print(x.shape)
    
    first_letter = file.split('/')[-1][0]

    # create output vector. If file begins with 'P', then y = [1,0], else y = [0,1], having the same dimension as x
    if first_letter == 'P':
        y = np.zeros((x.shape[0],2))
        y[:,0] = 1
    else:
        y = np.zeros((x.shape[0],2))
        y[:,1] = 1

    X.append(x)
    Y.append(y)

f = open('data_sizes.txt', 'a')

f.write('walk_exec_clean: ' + str(sum([x.shape[0] for x in X])) + '\n')

f.close()


['./execution_data_clean/P_1.mat', './execution_data_clean/P_10.mat', './execution_data_clean/P_2.mat', './execution_data_clean/P_3.mat', './execution_data_clean/P_4.mat', './execution_data_clean/P_5.mat', './execution_data_clean/P_9.mat', './execution_data_clean/S_1.mat', './execution_data_clean/S_10.mat', './execution_data_clean/S_2.mat', './execution_data_clean/S_3.mat', './execution_data_clean/S_4.mat', './execution_data_clean/S_5.mat', './execution_data_clean/S_9.mat']
(75, 14, 1000)
(48, 14, 1000)
(62, 14, 1000)
(77, 14, 1000)
(79, 14, 1000)
(85, 14, 1000)
(73, 14, 1000)
(58, 14, 1000)
(80, 14, 1000)
(64, 14, 1000)
(90, 14, 1000)
(89, 14, 1000)
(72, 14, 1000)
(98, 14, 1000)


In [3]:

kernels, chans, samples = 1, 14, 1000

Accuracies = []
p_acc = []
s_acc = []

for index_1 in range(0, 7):
    for index_2 in range(7,14):
        # build training set from allfiles expect the ones with index_1 and index_2
        X_train = [X[i] for i in range(len(X)) if i != index_1 and i != index_2]
        Y_train = [Y[i] for i in range(len(Y)) if i != index_1 and i != index_2]

        # build test set from the files with index_1 and index_2
        X_test = [X[i] for i in range(len(X)) if i == index_1 or i == index_2]
        Y_test = [Y[i] for i in range(len(Y)) if i == index_1 or i == index_2]

        X_train = np.concatenate(X_train)
        Y_train = np.concatenate(Y_train)

        # convert data to NHWC (trials, channels, samples, kernels) format.
        X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)

        model = EEGNet(nb_classes=2, Chans=chans, Samples=samples, dropoutRate=0.5, kernLength=32, F1=8, D=2, F2=16, dropoutType='Dropout')

        # compile the model and set the optimizers
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

        # count number of parameters in the model
        numParams    = model.count_params()     

        # set a valid path for the system to record model checkpoints
        # checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

        class_weights = {0:1, 1:1}

        fitted_model = model.fit(X_train, Y_train, batch_size = 32, epochs = 50, verbose = 2, 
                                class_weight=class_weights)
        
        #open file to write results in
        f = open("results_walk_exec_clean.txt", "a") 

        # make prediction on the 2 test files individually
        X_test_1 = X_test[0].reshape(X_test[0].shape[0], chans, samples, kernels)
        X_test_2 = X_test[1].reshape(X_test[1].shape[0], chans, samples, kernels)

        probs_1 = model.predict(X_test_1)
        probs_2 = model.predict(X_test_2)
        preds_1 = probs_1.argmax(axis = -1)
        preds_2 = probs_2.argmax(axis = -1)
        acc_1 = balanced_accuracy_score(Y_test[0].argmax(axis=-1), preds_1)
        acc_2 = balanced_accuracy_score(Y_test[1].argmax(axis=-1), preds_2)
        f.write("----------------------SELECTED FILES: %d and %d---------------------\n" % (index_1, index_2))
        f.write("Classification accuracy for P: %f \n" % (acc_1))
        f.write("Classification accuracy for S: %f \n" % (acc_2))

        p_acc.append(acc_1)
        s_acc.append(acc_2)

        # make prediction on the 2 test files together
        X_test = np.concatenate(X_test)
        X_test = X_test.reshape(X_test.shape[0], chans, samples, kernels)
        Y_test = np.concatenate(Y_test)


        probs       = model.predict(X_test)
        preds       = probs.argmax(axis = -1)
        acc         = balanced_accuracy_score(Y_test.argmax(axis=-1), preds)
        f.write("Classification accuracy for P and S: %f \n" % (acc))
        f.write("\n")

        f.close()

        Accuracies.append(acc)

print("Mean classification accuracy: %f " % (np.mean(Accuracies)))

# calculate the mean and std of the accuracies
p_acc = np.array(p_acc)
s_acc = np.array(s_acc)
acc = np.array(Accuracies)

f = open("results_walk_exec_clean.txt", "a")
f.write("Mean overall accuracy: %f \n" % (np.mean(acc)))
f.write("Std overall accuracy: %f \n" % (np.std(acc)))
f.write("Mean P accuracy: %f \n" % (np.mean(p_acc)))
f.write("Std P accuracy: %f \n" % (np.std(p_acc)))
f.write("Mean S accuracy: %f \n" % (np.mean(s_acc)))
f.write("Std S accuracy: %f \n" % (np.std(s_acc)))
f.close()



    

29/29 - 11s - loss: 0.3394 - accuracy: 0.9389 - 11s/epoch - 369ms/step
2/2 [==============================] - 0s 40ms/step


c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


5/5 [==============================] - 0s 42ms/step


c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


28/28 - 10s - loss: 0.3403 - accuracy: 0.9408 - 10s/epoch - 361ms/step
5/5 [==============================] - 0s 48ms/step
30/30 - 10s - loss: 0.3124 - accuracy: 0.9640 - 10s/epoch - 350ms/step
4/4 [==============================] - 0s 40ms/step
29/29 - 10s - loss: 0.3376 - accuracy: 0.9490 - 10s/epoch - 359ms/step
4/4 [==============================] - 0s 47ms/step
Mean classification accuracy: 0.979323 
